# PyTorch 基础：张量操作与自动微分

**目标**: 掌握 PyTorch 的核心概念和基础操作

**预计时间**: 60-90 分钟

---

## 📋 本 Notebook 内容

1. **张量基础**: 创建、操作、索引、形状变换
2. **自动微分**: autograd 机制与梯度计算
3. **神经网络模块**: nn.Module、层、损失函数
4. **优化器**: SGD、Adam、学习率调度
5. **完整训练流程**: 数据加载 → 模型定义 → 训练 → 评估
6. **实战案例**: MNIST 手写数字识别

---

## 环境设置

In [ ]:
# 导入必要的库
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

# 设置随机种子以保证可复现性
torch.manual_seed(42)
np.random.seed(42)

# 检测设备（GPU/CPU）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'PyTorch 版本: {torch.__version__}')
print(f'使用设备: {device}')
if device.type == 'cuda':
    print(f'GPU 名称: {torch.cuda.get_device_name(0)}')
    print(f'GPU 显存: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

---

## 第1部分: 张量（Tensor）基础

张量是 PyTorch 中的基本数据结构，类似于 NumPy 的数组，但可以在 GPU 上运行以加速计算。

### 1.1 创建张量

In [ ]:
# 从 Python 列表创建
t1 = torch.tensor([1, 2, 3, 4, 5])
print(f'从列表创建: {t1}')
print(f'数据类型: {t1.dtype}, 形状: {t1.shape}')

# 创建二维张量（矩阵）
t2 = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(f'\n二维张量:\n{t2}')
print(f'形状: {t2.shape}, 维度: {t2.ndim}')

# 从 NumPy 数组创建
arr = np.array([[1, 2], [3, 4], [5, 6]])
t3 = torch.from_numpy(arr)
print(f'\n从 NumPy 创建:\n{t3}')

# 特殊张量
zeros = torch.zeros(2, 3)  # 全0张量
ones = torch.ones(2, 3)    # 全1张量
rand = torch.rand(2, 3)    # 均匀分布随机数 [0, 1)
randn = torch.randn(2, 3)  # 标准正态分布随机数
arange = torch.arange(0, 10, 2)  # 等差数列
linspace = torch.linspace(0, 1, 5)  # 线性间隔

print(f'\n全0张量:\n{zeros}')
print(f'\n均匀分布随机数:\n{rand}')
print(f'\n等差数列: {arange}')
print(f'线性间隔: {linspace}')

### 1.2 张量操作

In [ ]:
# 算术运算
a = torch.tensor([1.0, 2.0, 3.0])
b = torch.tensor([4.0, 5.0, 6.0])

print(f'a + b = {a + b}')
print(f'a * b = {a * b}')  # 逐元素乘法
print(f'a ** 2 = {a ** 2}')

# 矩阵运算
A = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
B = torch.tensor([[5.0, 6.0], [7.0, 8.0]])

print(f'\n矩阵乘法 A @ B:\n{A @ B}')
print(f'\n转置 A.T:\n{A.T}')

# 聚合操作
x = torch.randn(3, 4)
print(f'\n张量 x:\n{x}')
print(f'求和: {x.sum()}')
print(f'均值: {x.mean()}')
print(f'最大值: {x.max()}')
print(f'沿轴0求和: {x.sum(dim=0)}')
print(f'沿轴1求均值: {x.mean(dim=1)}')

### 1.3 索引与切片

In [ ]:
# 创建示例张量
t = torch.arange(24).reshape(2, 3, 4)
print(f'原始张量 (2×3×4):\n{t}')

# 索引
print(f'\nt[0]: \n{t[0]}')
print(f'\nt[0, 1]: {t[0, 1]}')
print(f't[0, 1, 2]: {t[0, 1, 2]}')

# 切片
print(f'\nt[:, 0, :]: \n{t[:, 0, :]}')
print(f'\nt[0, :, 1:3]: \n{t[0, :, 1:3]}')

# 布尔索引
x = torch.randn(5)
print(f'\nx: {x}')
print(f'x > 0: {x > 0}')
print(f'x[x > 0]: {x[x > 0]}')

### 1.4 形状变换

In [ ]:
# 创建张量
x = torch.arange(12)
print(f'原始张量: {x}, 形状: {x.shape}')

# reshape: 改变形状（可能拷贝数据）
x_reshaped = x.reshape(3, 4)
print(f'\nreshape(3, 4):\n{x_reshaped}')

# view: 改变形状（共享内存）
x_view = x.view(2, 6)
print(f'\nview(2, 6):\n{x_view}')

# unsqueeze: 增加维度
x_unsqueeze = x.unsqueeze(0)
print(f'\nunsqueeze(0): 形状 {x_unsqueeze.shape}')

# squeeze: 移除大小为1的维度
y = torch.randn(1, 3, 1, 4)
print(f'\n原始形状: {y.shape}')
print(f'squeeze() 后: {y.squeeze().shape}')

# transpose: 转置
z = torch.randn(2, 3, 4)
print(f'\n原始形状: {z.shape}')
print(f'transpose(0, 1) 后: {z.transpose(0, 1).shape}')
print(f'permute(2, 0, 1) 后: {z.permute(2, 0, 1).shape}')

# flatten: 展平
flat = z.flatten()
print(f'\nflatten() 后: 形状 {flat.shape}')

### 1.5 GPU 加速

In [ ]:
# 创建张量
x_cpu = torch.randn(1000, 1000)
print(f'CPU 张量设备: {x_cpu.device}')

# 移动到 GPU（如果可用）
if torch.cuda.is_available():
    x_gpu = x_cpu.to(device)
    print(f'GPU 张量设备: {x_gpu.device}')
    
    # 性能对比
    import time
    
    # CPU 计算
    start = time.time()
    for _ in range(100):
        _ = x_cpu @ x_cpu
    cpu_time = time.time() - start
    
    # GPU 计算
    start = time.time()
    for _ in range(100):
        _ = x_gpu @ x_gpu
    torch.cuda.synchronize()  # 等待 GPU 完成
    gpu_time = time.time() - start
    
    print(f'\nCPU 时间: {cpu_time:.4f}s')
    print(f'GPU 时间: {gpu_time:.4f}s')
    print(f'加速比: {cpu_time / gpu_time:.2f}x')
else:
    print('GPU 不可用，跳过性能对比')

# CPU/GPU 之间转换
print(f'\n转换示例:')
x = torch.randn(3, 3).to(device)  # 创建并移动到设备
y = x.cpu()  # 移回 CPU
z = y.numpy()  # 转换为 NumPy（只能在 CPU 上）
print(f'x 设备: {x.device}')
print(f'y 设备: {y.device}')
print(f'z 类型: {type(z)}')

---

## 第2部分: 自动微分（Autograd）

PyTorch 的 autograd 机制可以自动计算梯度，是训练神经网络的核心。

### 2.1 梯度计算基础

In [ ]:
# 创建需要梯度的张量
x = torch.tensor([2.0], requires_grad=True)
print(f'x = {x}')
print(f'requires_grad: {x.requires_grad}')

# 前向传播：定义计算图
y = x ** 2 + 3 * x + 1
print(f'\ny = x^2 + 3x + 1 = {y}')

# 反向传播：计算梯度
y.backward()
print(f'\ndy/dx = 2x + 3 = {x.grad}')
print(f'在 x=2 时，dy/dx = {x.grad.item()}')

### 2.2 向量与矩阵的梯度

In [ ]:
# 向量梯度
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = x ** 2
z = y.sum()  # 标量输出

z.backward()
print(f'x = {x.data}')
print(f'y = x^2 = {y.data}')
print(f'z = sum(y) = {z.data}')
print(f'dz/dx = 2x = {x.grad}')

# 矩阵梯度
A = torch.tensor([[1.0, 2.0], [3.0, 4.0]], requires_grad=True)
B = (A ** 2).sum()

B.backward()
print(f'\nA:\n{A.data}')
print(f'dB/dA = 2A:\n{A.grad}')

### 2.3 计算图与链式法则

In [ ]:
# 多步计算
x = torch.tensor([3.0], requires_grad=True)

# y = 2x
y = 2 * x
print(f'y = 2x = {y.data}')

# z = y^2 = (2x)^2 = 4x^2
z = y ** 2
print(f'z = y^2 = {z.data}')

# w = z + 1 = 4x^2 + 1
w = z + 1
print(f'w = z + 1 = {w.data}')

# 反向传播
w.backward()

# 理论梯度：dw/dx = d(4x^2 + 1)/dx = 8x = 8*3 = 24
print(f'\ndw/dx = 8x = {x.grad}')
print(f'理论值 = 8 * 3 = 24')

# 链式法则演示
print(f'\n链式法则:')
print(f'dw/dx = (dw/dz) * (dz/dy) * (dy/dx)')
print(f'      = 1 * 2y * 2')
print(f'      = 2 * 2 * (2x)')
print(f'      = 8x')

### 2.4 梯度控制

In [ ]:
# 1. 停止梯度追踪
x = torch.tensor([2.0], requires_grad=True)
y = x ** 2

# detach() 从计算图中分离
y_detached = y.detach()
z = y_detached * 3

print(f'y.requires_grad: {y.requires_grad}')
print(f'y_detached.requires_grad: {y_detached.requires_grad}')
print(f'z.requires_grad: {z.requires_grad}')

# 2. torch.no_grad() 上下文管理器
x = torch.tensor([2.0], requires_grad=True)

with torch.no_grad():
    y = x ** 2
    print(f'\n在 no_grad 中，y.requires_grad: {y.requires_grad}')

# 3. 清零梯度
x = torch.tensor([1.0], requires_grad=True)
for i in range(3):
    y = x ** 2
    y.backward()
    print(f'迭代 {i+1}, x.grad: {x.grad}')
    x.grad.zero_()  # 清零梯度

### 2.5 梯度可视化

In [ ]:
# 可视化函数及其梯度
def visualize_gradient(func, func_name, x_range=(-3, 3)):
    """可视化函数及其梯度"""
    x_vals = torch.linspace(x_range[0], x_range[1], 100, requires_grad=True)
    y_vals = func(x_vals)
    
    # 计算梯度
    y_vals.backward(torch.ones_like(y_vals))
    grad_vals = x_vals.grad
    
    # 绘图
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # 函数曲线
    ax1.plot(x_vals.detach().numpy(), y_vals.detach().numpy(), linewidth=2)
    ax1.set_xlabel('x', fontsize=12)
    ax1.set_ylabel('f(x)', fontsize=12)
    ax1.set_title(f'函数: {func_name}', fontsize=14)
    ax1.grid(True, alpha=0.3)
    
    # 梯度曲线
    ax2.plot(x_vals.detach().numpy(), grad_vals.numpy(), color='red', linewidth=2)
    ax2.set_xlabel('x', fontsize=12)
    ax2.set_ylabel("f'(x)", fontsize=12)
    ax2.set_title(f'梯度: d/dx({func_name})', fontsize=14)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# 示例1: y = x^2
visualize_gradient(lambda x: x ** 2, 'x²')

# 示例2: y = sin(x)
visualize_gradient(lambda x: torch.sin(x), 'sin(x)')

# 示例3: y = x^3 - 3x^2 + 2
visualize_gradient(lambda x: x ** 3 - 3 * x ** 2 + 2, 'x³ - 3x² + 2')

---

## 第3部分: 神经网络模块（nn.Module）

PyTorch 提供了 `nn.Module` 类来构建神经网络，以及各种预定义的层和损失函数。

### 3.1 常用层

In [ ]:
# 1. 全连接层（Linear）
linear = nn.Linear(in_features=10, out_features=5)
x = torch.randn(3, 10)  # batch_size=3, features=10
output = linear(x)
print(f'Linear 层:')
print(f'  输入形状: {x.shape}')
print(f'  输出形状: {output.shape}')
print(f'  权重形状: {linear.weight.shape}')
print(f'  偏置形状: {linear.bias.shape}')

# 2. 卷积层（Conv2d）
conv = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
x = torch.randn(8, 3, 32, 32)  # batch_size=8, channels=3, height=32, width=32
output = conv(x)
print(f'\nConv2d 层:')
print(f'  输入形状: {x.shape}')
print(f'  输出形状: {output.shape}')

# 3. 池化层（MaxPool2d）
pool = nn.MaxPool2d(kernel_size=2, stride=2)
output_pooled = pool(output)
print(f'\nMaxPool2d 层:')
print(f'  输入形状: {output.shape}')
print(f'  输出形状: {output_pooled.shape}')

# 4. Batch Normalization
bn = nn.BatchNorm2d(16)
output_bn = bn(output)
print(f'\nBatchNorm2d 层:')
print(f'  输入形状: {output.shape}')
print(f'  输出形状: {output_bn.shape}')

# 5. Dropout
dropout = nn.Dropout(p=0.5)
x = torch.ones(10)
output_dropout = dropout(x)
print(f'\nDropout 层 (训练模式):')
print(f'  输入: {x}')
print(f'  输出: {output_dropout}')

### 3.2 激活函数

In [ ]:
# 常用激活函数
x = torch.linspace(-5, 5, 100)

activations = {
    'ReLU': F.relu(x),
    'Sigmoid': torch.sigmoid(x),
    'Tanh': torch.tanh(x),
    'LeakyReLU': F.leaky_relu(x, 0.1)
}

# 可视化
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.ravel()

for idx, (name, y) in enumerate(activations.items()):
    axes[idx].plot(x.numpy(), y.numpy(), linewidth=2)
    axes[idx].set_xlabel('x', fontsize=12)
    axes[idx].set_ylabel('f(x)', fontsize=12)
    axes[idx].set_title(name, fontsize=14)
    axes[idx].grid(True, alpha=0.3)
    axes[idx].axhline(y=0, color='k', linestyle='--', alpha=0.3)
    axes[idx].axvline(x=0, color='k', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

### 3.3 构建神经网络

In [ ]:
class SimpleNN(nn.Module):
    """简单的全连接神经网络"""
    
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        
        # 定义层
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        """前向传播"""
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 创建模型
model = SimpleNN(input_size=784, hidden_size=128, output_size=10)
print(model)

# 测试前向传播
x = torch.randn(32, 784)  # batch_size=32
output = model(x)
print(f'\n输入形状: {x.shape}')
print(f'输出形状: {output.shape}')

# 查看参数
print(f'\n模型参数:')
for name, param in model.named_parameters():
    print(f'  {name}: {param.shape}, requires_grad={param.requires_grad}')

# 计算总参数量
total_params = sum(p.numel() for p in model.parameters())
print(f'\n总参数量: {total_params:,}')

### 3.4 使用 Sequential 构建模型

In [ ]:
# 使用 Sequential 构建相同的模型
model_seq = nn.Sequential(
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

print('Sequential 模型:')
print(model_seq)

# 更复杂的示例
model_complex = nn.Sequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 10)
)

print('\n复杂 Sequential 模型:')
print(model_complex)

### 3.5 损失函数

In [ ]:
# 1. 交叉熵损失（分类）
criterion_ce = nn.CrossEntropyLoss()
predictions = torch.randn(5, 3)  # 5个样本，3个类别
targets = torch.tensor([0, 1, 2, 1, 0])  # 真实标签
loss_ce = criterion_ce(predictions, targets)
print(f'CrossEntropyLoss: {loss_ce.item():.4f}')

# 2. MSE 损失（回归）
criterion_mse = nn.MSELoss()
predictions = torch.randn(5, 1)
targets = torch.randn(5, 1)
loss_mse = criterion_mse(predictions, targets)
print(f'MSELoss: {loss_mse.item():.4f}')

# 3. BCE 损失（二分类）
criterion_bce = nn.BCEWithLogitsLoss()
predictions = torch.randn(5, 1)
targets = torch.randint(0, 2, (5, 1)).float()
loss_bce = criterion_bce(predictions, targets)
print(f'BCEWithLogitsLoss: {loss_bce.item():.4f}')

---

## 第4部分: 优化器与训练

优化器负责根据梯度更新模型参数。

### 4.1 常用优化器

In [ ]:
# 创建模型
model = SimpleNN(784, 128, 10)

# 1. SGD（随机梯度下降）
optimizer_sgd = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
print('SGD 优化器:')
print(optimizer_sgd)

# 2. Adam
optimizer_adam = optim.Adam(model.parameters(), lr=0.001)
print('\nAdam 优化器:')
print(optimizer_adam)

# 3. RMSprop
optimizer_rmsprop = optim.RMSprop(model.parameters(), lr=0.001)
print('\nRMSprop 优化器:')
print(optimizer_rmsprop)

# 不同学习率
optimizer_groups = optim.Adam([
    {'params': model.fc1.parameters(), 'lr': 0.001},
    {'params': model.fc2.parameters(), 'lr': 0.01}
])
print('\n不同层使用不同学习率:')
print(optimizer_groups)

### 4.2 学习率调度

In [ ]:
# 创建优化器
optimizer = optim.SGD(model.parameters(), lr=0.1)

# 1. StepLR: 每 step_size 个 epoch，lr *= gamma
scheduler_step = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# 2. MultiStepLR: 在指定 epoch，lr *= gamma
scheduler_multistep = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 20, 30], gamma=0.1)

# 3. ExponentialLR: lr *= gamma^epoch
scheduler_exp = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

# 4. CosineAnnealingLR: 余弦退火
scheduler_cosine = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

# 可视化学习率变化
def visualize_lr_schedule(scheduler, epochs=50):
    lrs = []
    optimizer = optim.SGD(model.parameters(), lr=0.1)
    
    if scheduler == 'step':
        sched = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
        title = 'StepLR (step_size=10, gamma=0.5)'
    elif scheduler == 'exp':
        sched = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
        title = 'ExponentialLR (gamma=0.95)'
    elif scheduler == 'cosine':
        sched = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
        title = f'CosineAnnealingLR (T_max={epochs})'
    
    for epoch in range(epochs):
        lrs.append(optimizer.param_groups[0]['lr'])
        sched.step()
    
    return lrs, title

# 绘制对比图
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, sched_type in enumerate(['step', 'exp', 'cosine']):
    lrs, title = visualize_lr_schedule(sched_type)
    axes[idx].plot(lrs, linewidth=2)
    axes[idx].set_xlabel('Epoch', fontsize=12)
    axes[idx].set_ylabel('Learning Rate', fontsize=12)
    axes[idx].set_title(title, fontsize=12)
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 第5部分: 完整训练流程

将所有组件整合，实现完整的训练和评估流程。

### 5.1 数据准备：MNIST 手写数字

In [ ]:
# 数据变换
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# 下载并加载数据
train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

print(f'训练集大小: {len(train_dataset)}')
print(f'测试集大小: {len(test_dataset)}')
print(f'图像形状: {train_dataset[0][0].shape}')

# 可视化部分数据
fig, axes = plt.subplots(2, 8, figsize=(16, 4))
axes = axes.ravel()

for i in range(16):
    image, label = train_dataset[i]
    axes[i].imshow(image.squeeze(), cmap='gray')
    axes[i].set_title(f'标签: {label}')
    axes[i].axis('off')

plt.suptitle('MNIST 训练数据样本', fontsize=16)
plt.tight_layout()
plt.show()

### 5.2 定义模型

In [ ]:
class MNISTNet(nn.Module):
    """MNIST 分类网络"""
    
    def __init__(self):
        super(MNISTNet, self).__init__()
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(64, 10)
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

# 创建模型
model = MNISTNet().to(device)
print(model)

# 参数统计
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'\n总参数量: {total_params:,}')
print(f'可训练参数: {trainable_params:,}')

### 5.3 训练与评估函数

In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    """训练一个 epoch"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(dataloader, desc='训练', leave=False)
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        # 前向传播
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 反向传播
        loss.backward()
        optimizer.step()
        
        # 统计
        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # 更新进度条
        pbar.set_postfix({
            'loss': running_loss / total,
            'acc': 100. * correct / total
        })
    
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc


def evaluate(model, dataloader, criterion, device):
    """评估模型"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc


def train_model(model, train_loader, test_loader, criterion, optimizer, 
                scheduler=None, epochs=10, device='cpu'):
    """完整训练流程"""
    history = {
        'train_loss': [],
        'train_acc': [],
        'test_loss': [],
        'test_acc': []
    }
    
    for epoch in range(epochs):
        # 训练
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        
        # 评估
        test_loss, test_acc = evaluate(model, test_loader, criterion, device)
        
        # 学习率调度
        if scheduler is not None:
            scheduler.step()
        
        # 记录历史
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['test_loss'].append(test_loss)
        history['test_acc'].append(test_acc)
        
        # 打印进度
        print(f'Epoch {epoch+1}/{epochs}: '
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, '
              f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')
    
    return history

### 5.4 开始训练

In [ ]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# 训练模型
print('开始训练...')
history = train_model(
    model=model,
    train_loader=train_loader,
    test_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    epochs=10,
    device=device
)

print('\n训练完成!')

### 5.5 可视化训练过程

In [ ]:
# 绘制训练曲线
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 损失曲线
epochs = range(1, len(history['train_loss']) + 1)
ax1.plot(epochs, history['train_loss'], 'o-', label='训练损失', linewidth=2)
ax1.plot(epochs, history['test_loss'], 's-', label='测试损失', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('损失', fontsize=12)
ax1.set_title('损失曲线', fontsize=14)
ax1.legend(fontsize=12)
ax1.grid(True, alpha=0.3)

# 准确率曲线
ax2.plot(epochs, history['train_acc'], 'o-', label='训练准确率', linewidth=2)
ax2.plot(epochs, history['test_acc'], 's-', label='测试准确率', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('准确率 (%)', fontsize=12)
ax2.set_title('准确率曲线', fontsize=14)
ax2.legend(fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f'\n最终测试准确率: {history["test_acc"][-1]:.2f}%')

### 5.6 预测可视化

In [ ]:
# 获取一批测试数据
dataiter = iter(test_loader)
images, labels = next(dataiter)
images = images[:16]
labels = labels[:16]

# 预测
model.eval()
with torch.no_grad():
    outputs = model(images.to(device))
    _, predicted = outputs.max(1)
    predicted = predicted.cpu()

# 可视化
fig, axes = plt.subplots(4, 4, figsize=(12, 12))
axes = axes.ravel()

for i in range(16):
    axes[i].imshow(images[i].squeeze(), cmap='gray')
    title = f'真实: {labels[i]} | 预测: {predicted[i]}'
    color = 'green' if labels[i] == predicted[i] else 'red'
    axes[i].set_title(title, color=color)
    axes[i].axis('off')

plt.suptitle('MNIST 预测结果（绿色=正确，红色=错误）', fontsize=16)
plt.tight_layout()
plt.show()

### 5.7 模型保存与加载

In [ ]:
# 保存模型
torch.save({
    'epoch': 10,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': history['test_loss'][-1],
    'accuracy': history['test_acc'][-1]
}, 'mnist_model.pth')

print('模型已保存到 mnist_model.pth')

# 加载模型
checkpoint = torch.load('mnist_model.pth')
model_loaded = MNISTNet().to(device)
model_loaded.load_state_dict(checkpoint['model_state_dict'])

print(f'\n模型已加载')
print(f'训练轮数: {checkpoint["epoch"]}')
print(f'测试损失: {checkpoint["loss"]:.4f}')
print(f'测试准确率: {checkpoint["accuracy"]:.2f}%')

# 验证加载的模型
test_loss, test_acc = evaluate(model_loaded, test_loader, criterion, device)
print(f'\n验证加载模型的准确率: {test_acc:.2f}%')

---

## 📝 总结

通过本 Notebook，我们全面学习了 PyTorch 的核心概念：

### 第1部分: 张量基础 ✅
- 创建张量的多种方式（从列表、NumPy、特殊张量）
- 张量操作（算术、矩阵运算、聚合）
- 索引与切片（布尔索引、多维索引）
- 形状变换（reshape、view、unsqueeze、squeeze、transpose）
- GPU 加速（CPU/GPU 数据转换、性能对比）

### 第2部分: 自动微分 ✅
- 梯度计算基础（requires_grad、backward）
- 向量与矩阵的梯度
- 计算图与链式法则
- 梯度控制（detach、no_grad、零梯度）
- 梯度可视化

### 第3部分: 神经网络模块 ✅
- 常用层（Linear、Conv2d、MaxPool2d、BatchNorm、Dropout）
- 激活函数（ReLU、Sigmoid、Tanh、LeakyReLU）
- 构建神经网络（nn.Module、Sequential）
- 损失函数（CrossEntropyLoss、MSELoss、BCELoss）

### 第4部分: 优化器 ✅
- 常用优化器（SGD、Adam、RMSprop）
- 学习率调度（StepLR、ExponentialLR、CosineAnnealingLR）

### 第5部分: 完整训练流程 ✅
- 数据加载（MNIST 数据集）
- 模型定义
- 训练与评估函数
- 训练过程可视化
- 预测结果展示
- 模型保存与加载

---

## 🎯 关键要点

1. **张量是 PyTorch 的基石**: 类似 NumPy，但支持 GPU 加速和自动微分
2. **autograd 自动计算梯度**: 无需手动推导，自动执行反向传播
3. **nn.Module 是模型基类**: 通过继承构建自定义网络
4. **训练循环**: 前向传播 → 计算损失 → 反向传播 → 更新参数
5. **GPU 加速**: 使用 `.to(device)` 将张量和模型移到 GPU

---

## 📚 下一步

- 🎯 深入学习 CNN：[03-cnn-image-classification.ipynb](03-cnn-image-classification.ipynb)
- 📖 探索 RNN：[04-rnn-text-classification.ipynb](04-rnn-text-classification.ipynb)
- 🚀 实战项目：[Stage 4 深度学习项目](../../docs/stage4/projects/)

---

**练习建议**:
1. 修改 MNIST 模型架构，尝试不同的层数和神经元数量
2. 尝试不同的优化器和学习率，观察收敛速度
3. 实现数据增强（旋转、缩放、噪声）
4. 添加早停（Early Stopping）机制
5. 可视化权重分布和梯度流